In [ ]:
pip install openpyxl

In [ ]:

import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import pandas as pd


# Global variables to store selected areas
column_one_area = None
column_two_area = None
heading_area = None

In [12]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import cv2
import numpy as np
import pandas as pd

# Global variables to store selected areas
column_one_area = None
column_two_area = None
heading_area = None

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image, column_one_area, column_two_area, heading_area

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    image_pil = Image.open(file_path)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

    # Reset the selected areas
    column_one_area = None
    column_two_area = None
    heading_area = None

# Function to reset the selected areas
def reset_selections():
    global column_one_area, column_two_area, heading_area

    # Set all areas to None
    column_one_area = None
    column_two_area = None
    heading_area = None

    # Update display
    update_display()

# Function to select column one area
def select_column_one_area():
    global column_one_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_one_area_click)

def on_column_one_area_click(event):
    global column_one_area

    # Save the starting point of the selection
    column_one_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_one_area_release)

def on_column_one_area_release(event):
    global column_one_area

    # Save the ending point of the selection
    column_one_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select column two area
def select_column_two_area():
    global column_two_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_two_area_click)

def on_column_two_area_click(event):
    global column_two_area

    # Save the starting point of the selection
    column_two_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_two_area_release)

def on_column_two_area_release(event):
    global column_two_area

    # Save the ending point of the selection
    column_two_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select heading area
def select_heading_area():
    global heading_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_heading_area_click)

def on_heading_area_click(event):
    global heading_area

    # Save the starting point of the selection
    heading_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_heading_area_release)

def on_heading_area_release(event):
    global heading_area

    # Save the ending point of the selection
    heading_area += (event.x, event.y)

    # Update display
    update_display()

# Function to update the display with selected areas
def update_display():
    global image_pil, column_one_area, column_two_area, heading_area

    # Create a copy of the original PIL image
    image_display = image_pil.copy()

    # Draw rectangles for selected areas on the image
    if column_one_area is not None:
        draw_rectangle(image_display, column_one_area)
    if column_two_area is not None:
        draw_rectangle(image_display, column_two_area)
    if heading_area is not None:
        draw_rectangle(image_display, heading_area)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_display)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)
    label_image.image = image_tk

# Function to draw a rectangle on an image
def draw_rectangle(image, area):
    draw = ImageDraw.Draw(image)
    draw.rectangle(area, outline='red', width=2)

# Function to perform OCR on selected areas
def perform_ocr():
    global image_pil, column_one_area, column_two_area, heading_area

    # Check if all areas are selected
    if column_one_area is None or column_two_area is None or heading_area is None:
        messagebox.showerror("Error", "Please select all areas before performing OCR.")
        return

    # Crop the selected areas from the image
    column_one_image = image_pil.crop(column_one_area)
    column_two_image = image_pil.crop(column_two_area)
    heading_image = image_pil.crop(heading_area)

    # Perform OCR on the cropped images
    column_one_text = pytesseract.image_to_string(column_one_image)
    column_two_text = pytesseract.image_to_string(column_two_image)
    heading_text = pytesseract.image_to_string(heading_image)

    # Display the OCR results in message boxes
    messagebox.showinfo("Column One Text", column_one_text)
    messagebox.showinfo("Column Two Text", column_two_text)
    messagebox.showinfo("Heading Text", heading_text)
    
# Function to save extracted text to a CSV file
def save_to_csv():
    global column_one_area, column_two_area, heading_area
    if column_one_area is None or column_two_area is None or heading_area is None:
        update_status_bar("Please select all areas before saving to CSV.")
        return

    try:
        # Extract text using pytesseract OCR
        column_one_text = pytesseract.image_to_string(column_one_area)
        column_two_text = pytesseract.image_to_string(column_two_area)
        heading_text = pytesseract.image_to_string(heading_area)

        # Create a CSV file and write extracted text to it
        with open("extracted_text.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Heading", "Column 1", "Column 2"])
            writer.writerow([heading_text, column_one_text, column_two_text])

        update_status_bar("Text saved to CSV successfully.")
    except Exception as e:
        update_status_bar(f"Error saving text to CSV: {e}")

# Create buttons for actions
button_open = tk.Button(root, text="Open Image", command=open_image)
button_open.pack()

button_select_column_one = tk.Button(root, text="Select Column One", command=select_column_one_area)
button_select_column_one.pack()

button_select_column_two = tk.Button(root, text="Select Column Two", command=select_column_two_area)
button_select_column_two.pack()

button_select_heading = tk.Button(root, text="Select Heading", command=select_heading_area)
button_select_heading.pack()

button_reset = tk.Button(root, text="Reset Selections", command=reset_selections)
button_reset.pack()

button_ocr = tk.Button(root, text="Perform OCR", command=perform_ocr)
button_ocr.pack()


# Start the tkinter event loop
root.mainloop()



In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from PIL import ImageGrab
import pytesseract

# Step 1: Capture screenshot from selected area
# Get coordinates of the selected area from user input or GUI interaction
# For example, using a GUI library like PyAutoGUI to capture screen coordinates
selected_area = (x1, y1, x2, y2)  # x1, y1, x2, y2 are the coordinates of the selected area

# Capture screenshot of the selected area
screenshot = ImageGrab.grab(bbox=selected_area) 

# Convert screenshot to OpenCV image
screenshot = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

# Step 2: Pre-process the screenshot
# Perform any necessary pre-processing on the screenshot image, such as resizing, cropping, or enhancing

# Step 3: Perform OCR on the screenshot
# Use Tesseract OCR library to extract text from the pre-processed screenshot
recognized_text = pytesseract.image_to_string(screenshot)

# Step 4: Extract and utilize the recognized text
# Process the recognized text as needed, such as saving to a file, displaying on the screen, etc.
print("Recognized Text: ", recognized_text)


In [ ]:
import pyautogui
from PIL import Image
import pytesseract
import tempfile

# Function to capture screenshot from selected area and perform OCR
def capture_screenshot_and_perform_ocr():
    # Get the coordinates of the selected area
    print("Select an area on the screen...")
    selected_area = pyautogui.screenshot()

    # Save the screenshot image to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_file:
        temp_file_path = temp_file.name
        selected_area.save(temp_file_path)

    # Perform OCR on the screenshot image
    ocr_text = pytesseract.image_to_string(Image.open(temp_file_path))

    # Print the OCR text
    print("OCR Text:")
    print(ocr_text)

# Call the capture_screenshot_and_perform_ocr function
capture_screenshot_and_perform_ocr()


In [ ]:
import pyautogui
from PIL import Image
import pytesseract
import tempfile
import tkinter as tk
from tkinter import filedialog, messagebox

# Global variables for storing the coordinates of the selected area
x1, y1, x2, y2 = 0, 0, 0, 0
drawing = False

# Function to capture screenshot from selected area and perform OCR
def capture_screenshot_and_perform_ocr():
    global x1, y1, x2, y2
    try:
        # Hide the main window temporarily
        root.withdraw()

        # Get the coordinates of the selected area
        selected_area = pyautogui.screenshot(region=(x1, y1, x2 - x1, y2 - y1))

        # Save the screenshot image to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_file:
            temp_file_path = temp_file.name
            selected_area.save(temp_file_path)

        # Perform OCR on the screenshot image
        ocr_text = pytesseract.image_to_string(Image.open(temp_file_path))

        # Print the OCR text
        print("OCR Text:")
        print(ocr_text)

        # Show a message box with the OCR text
        messagebox.showinfo("OCR Result", ocr_text)

        # Delete the temporary image file
        os.unlink(temp_file_path)

    except Exception as e:
        messagebox.showerror("Error", str(e))

    finally:
        # Reset the coordinates of the selected area
        x1, y1, x2, y2 = 0, 0, 0, 0

        # Show the main window again
        root.deiconify()

# Function to handle button click event
def on_button_click():
    capture_screenshot_and_perform_ocr()

# Function to handle Save button click event
def on_save_button_click():
    try:
        # Hide the main window temporarily
        root.withdraw()

        # Get the coordinates of the selected area
        selected_area = pyautogui.screenshot(region=(x1, y1, x2 - x1, y2 - y1))

        # Show a file dialog to save the screenshot
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                filetypes=[("PNG Files", "*.png"), ("All Files", "*.*")],
                                                title="Save Screenshot")

        if file_path:
            # Save the screenshot image to the specified file
            selected_area.save(file_path)
            messagebox.showinfo("Save Successful", "Screenshot saved successfully!")
        else:
            messagebox.showwarning("Save Cancelled", "Screenshot not saved.")

    except Exception as e:
        messagebox.showerror("Error", str(e))

    finally:
        # Reset the coordinates of the selected area
        x1, y1, x2, y2 = 0, 0, 0, 0

        # Show the main window again
        root.deiconify()

# Function to handle canvas click event
def on_canvas_click(event):
    global x1, y1, drawing
    if not drawing:
        x1, y1 = event.x, event.y
        drawing = True

# Function to handle canvas release event
def on_canvas_release(event):
    global x2, y2, drawing
    if drawing:
        x2, y2 = event.x, event.y
        drawing = False

        # Draw a rectangle on the canvas to show the selected area
        canvas.create_rectangle(x1, y1, x2, y2, outline='red', width=2)

# Function to handle canvas motion event
def on_canvas_motion(event):
    global x2, y2, drawing
    if drawing:
        x2, y2 = event.x, event.y
        canvas.delete("selection_box")
        canvas.create_rectangle(x1, y1, x2, y2, outline='red', width=2, tags="selection_box")

# Create main window
root = tk.Tk()
root.title("Screenshot OCR")

# Create canvas for screenshot selection
canvas = tk.Canvas(root, bg='white')
canvas.pack(expand=tk.YES, fill=tk.BOTH)

# Bind canvas events
canvas.bind("<Button-1>", on_canvas_click)
canvas.bind("<B1-Motion>", on_canvas_motion)
canvas.bind("<ButtonRelease-1>", on_canvas_release)

# Create capture and save buttons
capture_button = tk.Button(root, text="Capture Screenshot", command=on_button_click)
capture_button.pack(side=tk.LEFT, padx=10, pady=10)

save_button = tk.Button(root, text="Save Screenshot", command=on_save_button_click)
save_button.pack(side=tk.RIGHT, padx=10, pady=10)

# Start the main loop
root.mainloop()


In [ ]:
pip install pyautogui


In [ ]:
import pyautogui
from PIL import Image
import pytesseract
import tempfile
import tkinter as tk
from tkinter import filedialog, messagebox

# Global variables for storing the coordinates of the selected area
x1, y1, x2, y2 = 0, 0, 0, 0
drawing = False

# Function to capture screenshot from selected area and perform OCR
def capture_screenshot_and_perform_ocr():
    global x1, y1, x2, y2
    try:
        # Hide the main window temporarily
        root.withdraw()

        # Get the coordinates of the selected area
        selected_area = pyautogui.screenshot(region=(x1, y1, x2 - x1, y2 - y1))

        # Save the screenshot image to a temporary file
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_file:
            temp_file_path = temp_file.name
            selected_area.save(temp_file_path)

        # Perform OCR on the screenshot image
        ocr_text = pytesseract.image_to_string(Image.open(temp_file_path))

        # Print the OCR text
        print("OCR Text:")
        print(ocr_text)

        # Show a message box with the OCR text
        messagebox.showinfo("OCR Result", ocr_text)

        # Delete the temporary image file
        os.unlink(temp_file_path)

    except Exception as e:
        messagebox.showerror("Error", str(e))

    finally:
        # Reset the coordinates of the selected area
        x1, y1, x2, y2 = 0, 0, 0, 0

        # Show the main window again
        root.deiconify()

# Function to handle button click event
def on_button_click():
    capture_screenshot_and_perform_ocr()

# Function to handle Save button click event
def on_save_button_click():
    try:
        # Hide the main window temporarily
        root.withdraw()

        # Get the coordinates of the selected area
        selected_area = pyautogui.screenshot(region=(x1, y1, x2 - x1, y2 - y1))

        # Show a file dialog to save the screenshot
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                filetypes=[("PNG Files", "*.png"), ("All Files", "*.*")],
                                                title="Save Screenshot")

        if file_path:
            # Save the screenshot image to the specified file
            selected_area.save(file_path)
            messagebox.showinfo("Save Successful", "Screenshot saved successfully!")
        else:
            messagebox.showwarning("Save Cancelled", "Screenshot not saved.")

    except Exception as e:
        messagebox.showerror("Error", str(e))

    finally:
        # Reset the coordinates of the selected area
        x1, y1, x2, y2 = 0, 0, 0, 0

        # Show the main window again
        root.deiconify()

# Function to handle canvas click event
def on_canvas_click(event):
    global x1, y1, drawing
    if not drawing:
        x1, y1 = event.x, event.y
        drawing = True

# Function to handle canvas release event
def on_canvas_release(event):
    global x2, y2, drawing
    if drawing:
        x2, y2 = event.x, event.y
        drawing = False

        
        
        
        
        # Draw a rectangle on the canvas to show the selected area
        canvas.create_rectangle(x1, y1, x2, y2, outline="red")

        # Enable buttons
        btnCapture.config(state=tk.NORMAL)
        btnSave.config(state=tk.NORMAL)

# Function to handle canvas motion event
def on_canvas_motion(event):
    global x2, y2
    if drawing:
        # Update the coordinates of the second point of the rectangle while moving the mouse
        x2, y2 = event.x, event.y

# Create the main window
root = tk.Tk()
root.title("Screenshot OCR")

# Create a canvas to display the screenshot
canvas = tk.Canvas(root, bg="white")
canvas.pack(expand=tk.YES, fill=tk.BOTH)

# Create buttons
btnCapture = tk.Button(root, text="Capture Screenshot", command=on_button_click)
btnCapture.pack(side=tk.LEFT, padx=10, pady=10)
btnSave = tk.Button(root, text="Save Screenshot", command=on_save_button_click, state=tk.DISABLED)
btnSave.pack(side=tk.LEFT, padx=10, pady=10)

# Bind events to the canvas
canvas.bind("<Button-1>", on_canvas_click)

canvas.bind("<B1-Motion>", on_canvas_motion)
canvas.bind("<ButtonRelease-1>", on_canvas_release)

# Start the main loop
root.mainloop()


In [ ]:
import tkinter as tk
from PIL import ImageGrab, Image
from tkinter import messagebox, filedialog

# Global variables
x1, y1, x2, y2 = 0, 0, 0, 0
drawing = False

# Function to handle button click event
def on_button_click():
    global drawing
    drawing = True

# Function to handle canvas click event
def on_canvas_click(event):
    global x1, y1
    x1, y1 = event.x, event.y

# Function to handle canvas release event
def on_canvas_release(event):
    global x2, y2, drawing
    if drawing:
        x2, y2 = event.x, event.y
        drawing = False

        # Capture the screenshot of the selected area
        screenshot = ImageGrab.grab(bbox=(x1, y1, x2, y2))

        # Display the captured image
        screenshot.show()

        # Enable buttons
        btnSave.config(state=tk.NORMAL)

# Function to handle save button click event
def on_save_button_click():
    global x1, y1, x2, y2
    # Save the captured image to a file
    screenshot = ImageGrab.grab(bbox=(x1, y1, x2, y2))
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    if file_path:
        screenshot.save(file_path)
        messagebox.showinfo("Success", "Screenshot saved successfully!")

# Create the main window
root = tk.Tk()
root.title("Screenshot OCR")

# Create a canvas to display the screenshot
canvas = tk.Canvas(root, bg="white")
canvas.pack(expand=tk.YES, fill=tk.BOTH)

# Create buttons
btnCapture = tk.Button(root, text="Capture Screenshot", command=on_button_click)
btnCapture.pack(side=tk.LEFT, padx=10, pady=10)
btnSave = tk.Button(root, text="Save Screenshot", command=on_save_button_click, state=tk.DISABLED)
btnSave.pack(side=tk.LEFT, padx=10, pady=10)

# Bind events to the canvas
canvas.bind("<Button-1>", on_canvas_click)
canvas.bind("<B1-Motion>", on_canvas_release)
canvas.bind("<ButtonRelease-1>", on_canvas_release)

# Start the main loop
root.mainloop()


In [ ]:
from tkinter import *
from PIL import ImageGrab

# Global variables to store coordinates of selected area
x1, y1, x2, y2 = 0, 0, 0, 0
drawing = False

# Function to handle canvas click event
def on_canvas_click(event):
    global x1, y1, drawing
    x1, y1 = event.x, event.y
    drawing = True

# Function to handle canvas release event
def on_canvas_release(event):
    global x2, y2, drawing
    if drawing:
        x2, y2 = event.x, event.y
        drawing = False
        # Capture the screenshot of the selected area
        screenshot = ImageGrab.grab(bbox=(x1, y1, x2, y2))
        screenshot.show()

# Function to handle canvas motion event
def on_canvas_motion(event):
    global x2, y2
    if drawing:
        # Update the coordinates of the second point of the rectangle while moving the mouse
        x2, y2 = event.x, event.y

# Create the main window
root = Tk()
root.title("Snipping Tool")

# Create a canvas to display the screenshot
canvas = Canvas(root, bg="white")
canvas.pack(expand=YES, fill=BOTH)

# Bind events to the canvas
canvas.bind("<Button-1>", on_canvas_click)
canvas.bind("<B1-Motion>", on_canvas_motion)
canvas.bind("<ButtonRelease-1>", on_canvas_release)

# Start the main loop
root.mainloop()


In [ ]:


import os
import sys
import time
import json
import ctypes
import subprocess
from threading import Thread

import tkinter as tk
from tkinter import W, N, S, E, NW, LEFT
from tkinter import BooleanVar, messagebox, filedialog, ttk

import docx
from docx.shared import Inches

from PIL import Image, ImageGrab, ImageTk
from pynput import keyboard
from plyer import notification


class SnipperTool():

    def __init__(self,
                 master,
                 title='Snipper Tool',
                 width=500,
                 height=350,
                 dict_image_format={'PNG':'.png', 'BMP':'.bmp', 'JPEG':'.jpeg'},
                 dict_image_editor_application={'MS Paint': 'mspaint'}):

        self.master = master
        self.title = title
        self.width = width
        self.height = height

        self.dict_image_format = dict_image_format
        self.dict_image_editor_application = dict_image_editor_application

        self.control_padd = 10 # padding between widgets
        self.blank_col_padd = 50 # padding blank col

        self.text_box_dir_path = None
        self.btn_browse = None
        self.btn_start = None
        self.btn_create_docx = None
        self.btn_exit = None
        self.chk_box_save_setings = BooleanVar()

        self.logo = None
        self.bg_image = None
        self.path_icon = None
        self.path_settings_file = None

        self.cmb_box_image_ext = None
        self.cmb_box_image_editor = None

        self.thread_start_process = None
        self.show_image_captured_notification = True # No usage at the moment

        self._draw_window()


    # region Drawing Window

    def _draw_window(self):

        self.master.title(self.title)
        self._set_file_path()

        self._set_window_size_and_position()
        self._draw_layout()
        self._draw_icon_and_logo()
        self._write_header_and_instructions()
        self._draw_text_box()
        self._draw_buttons()
        self._draw_settings()

    def _set_window_size_and_position(self):
        # get screen width and height
        width_screen = self.master.winfo_screenwidth()
        height_screen = self.master.winfo_screenheight()

        # calculate position (x and y coordinates) for the Tk root window
        x_window_pos = (width_screen/2) - (self.width/2)
        y_window_pos = (height_screen/2) - (self.height/2)

        # set the dimensions & position of window
        self.master.geometry('%dx%d+%d+%d' % (self.width, self.height, x_window_pos, y_window_pos))
        self.master.resizable(width=False, height=False)


    def _draw_layout(self):
        for i in range(1, 6):
            self.master.columnconfigure(i, weight=1)
        for i in range(6):
            self.master.rowconfigure(i, weight=1)

        # 6 is for blank label
        padding_col = self.blank_col_padd - (self.control_padd/2) - 6
        # Left blank column
        lbl_pad = tk.Label(self.master, text="", )
        lbl_pad.grid(row=0, column=0, padx=(padding_col, 0), rowspan=11, sticky=W)
        # Right blank column
        lbl_pad = tk.Label(self.master, text="", )
        lbl_pad.grid(row=0, column=7, padx=(0, padding_col), rowspan=11, sticky=W)

    def _draw_icon_and_logo(self):
        if self.path_icon is not None and os.path.exists(self.path_icon):
            self.master.iconbitmap(self.path_icon)

            image_logo = (Image.open(self.path_icon)).resize((50, 50))
            self.logo = ImageTk.PhotoImage(image_logo)

            tk_label_logo = tk.Label(self.master)
            tk_label_logo.image = self.logo
            tk_label_logo['image'] = self.logo
            tk_label_logo.grid(row=0, column=2, columnspan=1, sticky=W+E,
                               padx=(self.control_padd/2, 0), pady=(self.control_padd, 0))

    def _write_header_and_instructions(self):
        # Header
        label_header = "Snipper Tool"
        header = tk.Label(self.master, text=label_header, font=("Arial", 25))
        header.grid(row=0, column=3, columnspan=4, sticky=W, pady=(self.control_padd, 0))

        # Instructions
        font_inst = ("Helvetica", 8)

        label_inst = "Instructions :\n" + \
                     "1: Click on the ' Browse ' button to select the folder.\n" + \
                     "2: Click on the ' Start ' button to start the process.\n" + \
                     "3: Press ' PrintScreen ' key to capture the screenshot.\n" + \
                     "4: Press ' Insert ' key to capture a screenshot & edit.\n" + \
                     "5: Click on ' Build Docx ' button to create a document.\n" + \
                     "    With images available in selected folder."

        label_inst = tk.Label(self.master, text=label_inst, font=font_inst,
                              justify=LEFT, wraplength=400, anchor=NW)
        label_inst.grid(row=5, column=1, columnspan=6, sticky=W,
                        padx=self.control_padd/2, pady=(0, self.control_padd/2))

    def _draw_text_box(self):
        self.text_box_dir_path = tk.Entry(self.master, bd=2, width=36)
        self.text_box_dir_path.grid(row=1, column=1, columnspan=4, sticky=W+E,
                                    padx=self.control_padd/2, pady=(self.control_padd/2, 0))
        self.text_box_dir_path.delete(0, tk.END)

    def _draw_buttons(self):

        self.btn_browse = tk.Button(self.master, text="Browse", width=10, command=self._browse_dir)
        self.btn_browse.grid(row=1, column=5, sticky=W+N+E+S,
                             padx=self.control_padd/2, pady=self.control_padd/2)

        self.btn_start = tk.Button(self.master, text="Start", command=self._start_capture_process)
        self.btn_start.grid(row=3, column=1, columnspan=2, sticky=W+N+E+S,
                            padx=self.control_padd/2, pady=self.control_padd/2)

        self.btn_create_docx = tk.Button(self.master, text="Create Docx",
                                         command=self._create_document)
        self.btn_create_docx.grid(row=3, column=3, columnspan=2, sticky=W+N+E+S,
                                  padx=self.control_padd/2, pady=self.control_padd/2)

        self.btn_exit = tk.Button(self.master, text="Exit", width=10, command=self.exit)
        self.btn_exit.grid(row=3, column=5, sticky=W+N+E+S,
                           padx=self.control_padd/2, pady=self.control_padd/2)

    def _draw_settings(self):
        font_text = ('Callibri', '8')
        # Image extension type drop-down
        label_type = tk.Label(self.master, text="Type:", anchor=W)
                                                # "Editor:"
        label_type.grid(row=2, column=1, sticky=W,
                        padx=(self.control_padd/2, 0), pady=self.control_padd/2)

        self.cmb_box_image_ext = ttk.Combobox(self.master, width=8,
                                              state='readonly', font=font_text,
                                              values=list(self.dict_image_format.keys()))
        self.cmb_box_image_ext.grid(row=2, column=2, sticky=W+E,
                                    padx=(0, self.control_padd/2),
                                    pady=self.control_padd/2)

        # Image editor type drop-down
        label_editor = tk.Label(self.master, text="Editor:", )
        label_editor.grid(row=2, column=3, sticky=W,
                          padx=(self.control_padd/2, 0), pady=self.control_padd/2)

        image_editors = list(self.dict_image_editor_application.keys())
        self.cmb_box_image_editor = ttk.Combobox(self.master, width=7, state='readonly',
                                                 font=font_text, values=image_editors)
        self.cmb_box_image_editor.grid(row=2, column=4, sticky=W+E,
                                       padx=(0, self.control_padd/2),
                                       pady=self.control_padd/2)

        # Save config check box
        chk_box_save = tk.Checkbutton(self.master, text='Save config',
                                      variable=self.chk_box_save_setings,
                                      onvalue=True, offvalue=False, command=self._save_setting)
        chk_box_save.grid(row=2, column=5, sticky=W,
                          padx=self.control_padd/2, pady=self.control_padd/2)

        self._set_default_settings_value()

    # endregion Drawing Window


    # region Settings

    def _set_file_path(self):
        assets_path = os.path.join(os.getcwd(), 'assets')

        if not os.path.exists(assets_path):
            os.mkdir(assets_path)
        self.path_icon = os.path.join(assets_path, 'SnipperIcon.ico')

        # For executable or installer to have write access
        # Creating settings in user APPDATA folder
        path_settings = os.path.join(os.getenv('APPDATA'), self.title)
        if not os.path.exists(path_settings):
            os.mkdir(path_settings)
        self.path_settings_file = os.path.join(path_settings, 'settings.json')

    def _set_default_settings_value(self):
        self.cmb_box_image_ext.current(0)
        self.cmb_box_image_editor.current(0)
        self.chk_box_save_setings.set(False)

        self._load_settings()

        self.cmb_box_image_ext.bind("<<ComboboxSelected>>", self._save_setting)
        self.cmb_box_image_editor.bind("<<ComboboxSelected>>", self._save_setting)

    def _load_settings(self):
        if os.path.exists(self.path_settings_file):
            settings = None
            with open(self.path_settings_file, "r") as read_file:
                settings = json.load(read_file)

            list_keys = list(self.dict_image_format.keys())
            if 'image_type' in settings and settings['image_type'] in list_keys:
                self.cmb_box_image_ext.current(list_keys.index(settings['image_type']))

            list_keys = list(self.dict_image_editor_application.keys())
            if 'image_editor' in settings and settings['image_editor'] in list_keys:
                self.cmb_box_image_editor.current(list_keys.index(settings['image_editor']))

            if 'save_config' in settings and isinstance(settings['save_config'], bool):
                self.chk_box_save_setings.set(settings['save_config'])

    def _save_setting(self, event=None):
        try:
            if self.chk_box_save_setings.get():
                settings = {'image_type': self.cmb_box_image_ext.get(),
                            'image_editor': self.cmb_box_image_editor.get(),
                            'save_config': self.chk_box_save_setings.get()}
                with open(self.path_settings_file, "w") as write_file:
                    json.dump(settings, write_file)
            else:
                if os.path.exists(self.path_settings_file):
                    os.remove(self.path_settings_file)
        except Exception as error:
            messagebox.showinfo(self.title, str(error))

    # endregion Settings


    def _browse_dir(self):
        dir_path = filedialog.askdirectory()
        self.text_box_dir_path.delete(0, tk.END)
        self.text_box_dir_path.insert(0, dir_path)


    # region Capture Process

    def _start_capture_process(self):
        if os.path.exists(self.text_box_dir_path.get()) and self.thread_start_process is None:
            self.thread_start_process = Thread(target=self._listen_key_events, daemon=True)
            self.thread_start_process.start()

            messagebox.showinfo(self.title, 'Process started.')
            self.master.iconify() # Minimises window tool
        elif not os.path.exists(self.text_box_dir_path.get()):
            messagebox.showinfo(self.title, "Please select correct path.")
        elif self.thread_start_process is not None:
            self._notify('Process already running.', 3, self.title, self.path_icon)

    def _listen_key_events(self):
        with keyboard.Listener(on_release=self._on_key_relase) as listener:
            listener.join()

    def _on_key_relase(self, key):
        if key == keyboard.Key.print_screen:
            # self._capture_screen(open_image=False)
            thread = Thread(target=self._capture_screen, args=(False, ), daemon=True)
            thread.start()
        elif key == keyboard.Key.insert:
            # self._capture_screen(open_image=True)
            thread = Thread(target=self._capture_screen, args=(True, ), daemon=True)
            thread.start()

    def _capture_screen(self, open_image=False, prefix_name='Screen_Shot_'):
        image_file_extn = self.dict_image_format[self.cmb_box_image_ext.get()]
        image_file_name = prefix_name + self.time_stamp() + image_file_extn
        image_file_path = os.path.join(self.text_box_dir_path.get(), image_file_name)

        ImageGrab.grab().save(image_file_path)

        if self.show_image_captured_notification:
            message = 'Image captured ' + image_file_name
            self._notify(message, 2, self.title, self.path_icon)

        if open_image:
            app_name = self.dict_image_editor_application[self.cmb_box_image_editor.get()]
            open_image_in_editor_cmd = [app_name, os.path.abspath(image_file_path)]
            subprocess.Popen(open_image_in_editor_cmd, shell=True)

    # endregion Capture Process


    # region Create Document

    def _create_document(self):
        """Creates word document wiht images present in selected directory"""

        if not os.path.exists(self.text_box_dir_path.get()):
            messagebox.showinfo(self.title, 'Please select correct path.')
            return

        list_path_images_in_dir = self._get_list_path_images()

        # If any images in selected directory
        if list_path_images_in_dir:
            doc = docx.Document()

            # Setting page layout of 1 inch margin from all sides
            for section in doc.sections:
                section.top_margin, section.bottom_margin = Inches(1), Inches(1)
                section.left_margin, section.right_margin = Inches(1), Inches(1)

            # Adding images to word document
            for index, file_path in enumerate(list_path_images_in_dir):
                doc.add_heading(str(index + 1), 4).style = 'List'
                doc.add_picture(file_path, width=Inches(6.5), height=Inches(3.65))
                doc.add_paragraph('') # For new line

            # Saving word document
            doc_name = 'Document_' + self.time_stamp() + '.docx'
            path_doc = os.path.abspath(os.path.join(self.text_box_dir_path.get(), doc_name))
            doc.save(path_doc)

            # Show messsage
            count_images = str(len(list_path_images_in_dir))
            msg = "Word document created with " + count_images + " images at " + path_doc + '.'
            messagebox.showinfo(self.title, msg)
        else:
            messagebox.showinfo(self.title, "No images present in selected directory.")

    def _get_list_path_images(self):
        list_path_images_in_dir = []
        image_extensions = tuple(list(self.dict_image_format.values()) + ['.jpg'])

        # Finding all images present in selected directory
        for file_name in os.listdir(self.text_box_dir_path.get()):
            if file_name.lower().endswith(image_extensions):
                path_image = os.path.abspath(os.path.join(self.text_box_dir_path.get(), file_name))
                list_path_images_in_dir.append(path_image)

        return list_path_images_in_dir

    # endregion Create Document


    def exit(self):
        """To close the application"""
        self.master.destroy()


    def time_stamp(self):
        """Returns time stamp with custom format as string"""
        return str(time.strftime("%Y-%m-%d-%H-%M-%S"))


    def _notify(self, message: str, timeout: int, title: str, app_icon: str) -> None:
        """
        Shows notification message
        Args:
            message (str):  the message to display
            timeout (int):  the time for which notification should be visible
            title (str):    title of notification
            app_icon (str): accessible path of icon file
        """
        notification.notify(title=title, message=message,
                            app_name=title, app_icon=app_icon,
                            timeout=timeout)


# DPI aware
if 'win' in sys.platform:
    ctypes.windll.shcore.SetProcessDpiAwareness(1)


if __name__ == "__main__":
    MASTER = tk.Tk()
    SnipperTool(MASTER)
    MASTER.mainloop()


In [ ]:
pip install exceptions

In [ ]:
import cv2
import numpy as np
import pandas as pd


img = cv2.imread('image.png',0)

#fn to show np images with cv2 and close on any key press
def imshow(img, label='default'):
    cv2.imshow(label, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 250, 255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
#inverting the image
img_bin = 255-img_bin 

# Defining a kernel length
kernel_length = np.array(img).shape[1]//80

# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
#cv2.imwrite("verticle_lines.jpg",verticle_lines_img)

# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
#cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)

# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5

beta = 1.0 - alpha# This function helps to add two image with specific weight parameter to get a third image as summation of two image.

img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)

img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)

(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

cv2.imwrite("img_final_bin.jpg",img_final_bin)


# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

""" this section saves each extracted box as a seperate image."""

idx = 0
for c in contours:
    # Returns the location and width,height for every contour
    x, y, w, h = cv2.boundingRect(c)
    #only selecting boxes within certain width height range
    if (w > 10 and h > 15 and h < 50):
        idx += 1
        new_img = img[y:y+h, x:x+w]
        #cv2.imwrite("kanan/1/"+ "{}-{}-{}-{}".format(x, y, w, h) + '.jpg', new_img)


# Extract bounding boxes for contours and store in boundingBoxes variable
boundingBoxes = []
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    boundingBoxes.append((x, y, w, h))

#get set of all y-coordinates to sort boxes row wise
def getsety(boxes):
    ally = []
    for b in boxes:
        ally.append(b[1])
    ally = set(ally)
    ally = sorted(ally)
    return ally

#sort boxes by y in certain range, because if image is tilted than same row boxes 
#could have different Ys but within certain range
def sort_boxes(boxes, y, row_column):
    l = []
    for b in boxes:
        if (b[2] > 10 and b[3] > 15 and b[3] < 50):
            if b[1] >= y - 7 and b[1] <= y + 7:
                l.append(b)

    if l in row_column:
        return row_column
    else:
        row_column.append(l)
        return row_column

#sort each row using X of each box to sort it column wise
def sortrows(rc):
    new_rc = []
    for row in rc:
        r_new = sorted(row, key = lambda cell: cell[0])
        new_rc.append(r_new)
    return new_rc

row_column = []
for i in getsety(boundingBoxes):
    row_column = sort_boxes(boundingBoxes, i, row_column)
row_column = [i for i in row_column if i != []]

#final np array with sorted boxes from top left to bottom right
row_column = sortrows(row_column)




In [ ]:
import cv2
import pytesseract
import pandas as pd

# Load the image
img = cv2.imread('image.png', 0)

# Perform image processing operations as mentioned in your code

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['Box', 'Text'])

# Loop through each box in the sorted boxes
for box in sorted_boxes:
    x, y, w, h = box
    roi = img[y:y+h, x:x+w]  # Extract the region of interest (ROI) from the image
    text = pytesseract.image_to_string(roi)  # Extract text from ROI using Tesseract OCR
    box_str = f'({x},{y})'  # Convert box coordinates to string
    df = df.append({'Box': box_str, 'Text': text}, ignore_index=True)  # Append to DataFrame

# Display the resulting DataFrame
print(df)


In [ ]:
pip install tesserocr


In [ ]:
pip install pytesseract

In [ ]:
pip install pandas


In [ ]:
pip install opencv-python


In [ ]:
import os
import platform
from typing import List, Tuple

from tesserocr import PyTessBaseAPI, iterate_level, RIL

system = platform.system()
if system == 'Linux':
    tessdata_folder_default = ''
elif system == 'Windows':
    tessdata_folder_default = r'C:\Program Files (x86)\Tesseract-OCR\tessdata'
else:
    raise NotImplementedError

# this tesseract specific env variable takes precedence for tessdata folder location selection
# especially important for windows, as we don't know if we're running 32 or 64bit tesseract
tessdata_folder = os.getenv('TESSDATA_PREFIX', tessdata_folder_default)


def get_layout_boxes(input_image,  # PIL image object
                     level: RIL,
                     include_text: bool,
                     include_boxes: bool,
                     language: str,
                     psm: int,
                     tessdata_path='') -> List[Tuple]:
    """
    Get image components coordinates. It will return also text if include_text is True.
    :param input_image: input PIL image
    :param level: page iterator level, please see "RIL" enum
    :param include_text: if True return boxes texts
    :param include_boxes: if True return boxes coordinates
    :param language: language for OCR
    :param psm: page segmentation mode, by default it is PSM.AUTO which is 3
    :param tessdata_path: the path to the tessdata folder
    :return: list of tuples: [((x1, y1, x2, y2), text)), ...]
    """
    assert any((include_text, include_boxes)), (
        'Both include_text and include_boxes can not be False.')

    if not tessdata_path:
        tessdata_path = tessdata_folder

    try:
        with PyTessBaseAPI(path=tessdata_path, lang=language) as api:
            api.SetImage(input_image)

            api.SetPageSegMode(psm)
            api.Recognize()
            page_iterator = api.GetIterator()
            data = []
            for pi in iterate_level(page_iterator, level):
                bounding_box = pi.BoundingBox(level)
                if bounding_box is not None:
                    text = pi.GetUTF8Text(level) if include_text else None
                    box = bounding_box if include_boxes else None
                    data.append((box, text))
            return data
    except RuntimeError:
        print('Please specify correct path to tessdata.')

In [ ]:

"""Cell recognition algorithm."""
import numpy as np
import cv2
from imutils import resize

# demo document
dem = dict(
    img_path='image.png',
    gt_boxes=np.array([[451,  67, 749, 749]]),
    in_boxes=np.array([[455,  84, 760, 785]]))

dem_wdth = dem['in_boxes'][0][3] - dem['in_boxes'][0][1]
dem_hght = dem['in_boxes'][0][2] - dem['in_boxes'][0][0]
dem_xmin = dem['in_boxes'][0][1]
dem_ymin = dem['in_boxes'][0][0]

dem_image = cv2.imread(dem['img_path'])

# detected table from document
tbl_image = dem_image[dem_ymin: dem_ymin + dem_hght,
                      dem_xmin: dem_xmin + dem_wdth]

# threshold and resize table image
tbl_gray = cv2.cvtColor(tbl_image, cv2.COLOR_BGR2GRAY)
tbl_thresh_bin = cv2.threshold(tbl_gray, 127, 255, cv2.THRESH_BINARY)[1]

R = 2.5
tbl_resized = resize(tbl_thresh_bin, width=int(tbl_image.shape[1] // R))


def get_dividers(img, axis):
    """Return array indicies of white horizontal or vertical lines."""
    blank_lines = np.where(np.all(img == 255, axis=axis))[0]
    filtered_idx = np.where(np.diff(blank_lines) != 1)[0]
    return blank_lines[filtered_idx]


dims = tbl_image.shape[0], tbl_image.shape[1]

# table mask to search for gridlines
tbl_str = np.zeros(dims, np.uint8)
tbl_str = cv2.rectangle(tbl_str, (0, 0), (dims[1] - 1, dims[0] - 1), 255, 1)

for a in [0, 1]:
    dividers = get_dividers(tbl_resized, a)
    start_point = [0, 0]
    end_point = [dims[1], dims[1]]
    for i in dividers:
        i *= R
        start_point[a] = int(i)
        end_point[a] = int(i)
        cv2.line(tbl_str,
                 tuple(start_point),
                 tuple(end_point),
                 255,
                 1)


contours, hierarchy = cv2.findContours(tbl_str,
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)


def sort_contours(cnts, method="left-to-right"):
    """Return sorted countours."""
    reverse = False
    k = 0
    if method in ['right-to-left', 'bottom-to-top']:
        reverse = True
    if method in ['top-to-bottom', 'bottom-to-top']:
        k = 1
    b_boxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, b_boxes) = zip(*sorted(zip(cnts, b_boxes),
                                  key=lambda b: b[1][k],
                                  reverse=reverse))
    return (cnts, b_boxes)


contours, boundingBoxes = sort_contours(contours, method='top-to-bottom')

# remove countours of the whole table
bb_filtered = [list(t) for t in boundingBoxes
               if t[2] < dims[1] and t[3] < dims[0]]

# allocate countours in table-like structure
rows = []
columns = []

for i, bb in enumerate(bb_filtered):
    if i == 0:
        columns.append(bb)
        previous = bb
    else:
        if bb[1] < previous[1] + previous[3]/2:
            columns.append(bb)
            previous = bb
            if i == len(bb_filtered) - 1:
                rows.append(columns)
        else:
            rows.append(columns)
            columns = []
            previous = bb
            columns.append(bb)
            
            
            
            
import pandas as pd

# Create an empty dataframe to store the final table data
table_data = pd.DataFrame()

# Loop through each row in the rows list
for row in rows:
    # Sort the bounding boxes of the cells in the row from left to right
    sorted_row = sorted(row, key=lambda x: x[0])

    # Extract the cell data from the sorted bounding boxes
    cell_data = []
    for bb in sorted_row:
        xmin, ymin, w, h = bb
        cell_image = tbl_image[ymin:ymin+h, xmin:xmin+w]
        cell_text = "Extracted cell data" # Replace this with your code to extract the text from the cell image
        cell_data.append(cell_text)
    
    # Add the extracted cell data as a row to the table_data dataframe
    table_data = table_data.append(pd.Series(cell_data), ignore_index=True)

# Print the final dataframe
print(table_data)



In [ ]:
rows

In [ ]:
pip install imutils

In [ ]:
import cv2
import numpy as np
import pandas as pd
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # Replace with the correct path

img = cv2.imread('image.png',0)

# Function to show np images with cv2 and close on any key press
def imshow(img, label='default'):
    cv2.imshow(label, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 250, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# Inverting the image
img_bin = 255 - img_bin

# Defining a kernel length
kernel_length = np.array(img).shape[1] // 80

# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)

# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha

# This function helps to add two images with specific weight parameter to get a third image as summation of two images.
img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)

img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)

(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

cv2.imwrite("img_final_bin.jpg", img_final_bin)

# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# This function returns a set of all y-coordinates to sort boxes row-wise
def get_set_y(boxes):
    all_y = []
    for b in boxes:
        y_values = b[1][:, 1]  # Extract the second column (y-values) from the numpy array
        for y in y_values:
            all_y.append(int(y))  # Convert y-value to integer and append to the list
    all_y = set(all_y)
    all_y = sorted(all_y)
    return all_y



# This function sorts boxes by y in a certain range, because if the image is tilted then same row boxes 
# could have different Ys but within a certain range
def sort_boxes(boxes, y, row_column):
    l = []
    for b in boxes:
        if (np.all(b[2] > 10) and np.all(b[3] > 15) and np.all(b[3] < 50)):
            if np.all(b[1] >= y - 7) and np.all(b[1] <= y + 7):
                l.append(b)
    l = sorted(l, key=lambda x: x[0])
    row_column.append(l)
    return row_column


# Sort each row using X of each box to sort it column-wise
def sort_rows(rc):
    new_rc = []
    for row in rc:
        r_new = sorted(row, key=lambda cell: cell[0])
        new_rc.append(r_new)
    return new_rc

row_column = []
for i in get_set_y(contours):
    row_column = sort_boxes(contours, i, row_column)
row_column = [list(i) for i in row_column if i != []]

# Final np array with sorted boxes from top left to bottom right
row_column = sort_rows(row_column)

# OCR to read text from the boxes
import pytesseract

# Create a dataframe to store the extracted text values
df = pd.DataFrame(columns=['Text', 'X', 'Y', 'Width', 'Height'])

for row in row_column:
    for cell in row:
        x, y, w, h = cell
        cropped_img = img[y:y + h, x:x + w]
        text = pytesseract.image_to_string(cropped_img, lang='eng')
        df = df.append({'Text': text, 'X': x, 'Y': y, 'Width': w, 'Height': h}, ignore_index=True)

# Display the dataframe
print(df)


In [ ]:
text

In [ ]:
pip install img2table


In [ ]:
from img2table.document import Image

# Instantiation of the image
img = Image(src="image.png")

# Table identification
img_tables = img.extract_tables()

# Result of table identification
img_tables

[ExtractedTable(title=None, bbox=(10, 8, 745, 314),shape=(6, 3)),
 ExtractedTable(title=None, bbox=(936, 9, 1129, 111),shape=(2, 2))]

In [ ]:
img_tables

In [ ]:
import cv2
import pytesseract
import pandas as pd

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


def ocr_table_to_excel(image_path, excel_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to the grayscale image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 5)

    # Perform OCR using Tesseract
    custom_config = r'--oem 3 --psm 6'
    ocr_text = pytesseract.image_to_string(thresh, config=custom_config)

    # Extract the OCR text into a list of lines
    lines = ocr_text.split("\n")

    # Initialize empty lists to store row and column data
    rows = []

    # Loop through each line of OCR text
    for line in lines:
        # Split the line into individual words
        words = line.split()

        # If there are words in the line
        if len(words) > 0:
            # Add the line as a row in the rows list
            rows.append(words)

    # Create a Pandas DataFrame from the rows
    df = pd.DataFrame(rows)
    return df
    # Write the DataFrame to an Excel file
#     df.to_excel(excel_path, index=False, header=False)

    print("OCR table data has been extracted and saved to Excel successfully!")

# Example usage
image_path = "image.png"  # Path to the input image
excel_path = "output.xlsx"  # Path to the output Excel file
ocr_table_to_excel(image_path, excel_path)


In [ ]:
df

In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# demo document
dem = dict(
    img_path='image.png',
    gt_boxes=np.array([[451,  67, 749, 749]]),
    in_boxes=np.array([[455,  84, 760, 785]]))

dem_wdth = dem['in_boxes'][0][3] - dem['in_boxes'][0][1]
dem_hght = dem['in_boxes'][0][2] - dem['in_boxes'][0][0]
dem_xmin = dem['in_boxes'][0][1]
dem_ymin = dem['in_boxes'][0][0]

dem_image = cv2.imread(dem['img_path'])

# detected table from document
tbl_image = dem_image[dem_ymin: dem_ymin + dem_hght,
                      dem_xmin: dem_xmin + dem_wdth]

# threshold and resize table image
tbl_gray = cv2.cvtColor(tbl_image, cv2.COLOR_BGR2GRAY)
tbl_thresh_bin = cv2.threshold(tbl_gray, 127, 255, cv2.THRESH_BINARY)[1]

R = 2.5
tbl_resized = resize(tbl_thresh_bin, width=int(tbl_image.shape[1] // R))


def get_dividers(img, axis):
    """Return array indicies of white horizontal or vertical lines."""
    blank_lines = np.where(np.all(img == 255, axis=axis))[0]
    filtered_idx = np.where(np.diff(blank_lines) != 1)[0]
    return blank_lines[filtered_idx]


dims = tbl_image.shape[0], tbl_image.shape[1]

# table mask to search for gridlines
tbl_str = np.zeros(dims, np.uint8)
tbl_str = cv2.rectangle(tbl_str, (0, 0), (dims[1] - 1, dims[0] - 1), 255, 1)

for a in [0, 1]:
    dividers = get_dividers(tbl_resized, a)
    start_point = [0, 0]
    end_point = [dims[1], dims[1]]
    for i in dividers:
        i *= R
        start_point[a] = int(i)
        end_point[a] = int(i)
        cv2.line(tbl_str,
                 tuple(start_point),
                 tuple(end_point),
                 255,
                 1)


contours, hierarchy = cv2.findContours(tbl_str,
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)


def sort_contours(cnts, method="left-to-right"):
    """Return sorted countours."""
    reverse = False
    k = 0
    if method in ['right-to-left', 'bottom-to-top']:
        reverse = True
    if method in ['top-to-bottom', 'bottom-to-top']:
        k = 1
    b_boxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, b_boxes) = zip(*sorted(zip(cnts, b_boxes),
                                  key=lambda b: b[1][k],
                                  reverse=reverse))
    return (cnts, b_boxes)


contours, boundingBoxes = sort_contours(contours, method='top-to-bottom')

# remove countours of the whole table
bb_filtered = [list(t) for t in boundingBoxes
               if t[2] < dims[1] and t[3] < dims[0]]

# allocate countours in table-like structure
rows = []
columns = []

for i, bb in enumerate(bb_filtered):
    if i == 0:
        columns.append(bb)
        previous = bb
    else:
        if bb[1] < previous[1] + previous[3]/2:
            columns.append(bb)
            previous = bb
            if i == len(bb_filtered) - 1:
                rows.append(columns)
        else:
            rows.append(columns)
            columns = []
            previous = bb
            columns.append(bb)

import pandas as pd

# Create an empty dataframe to store the final table data
table_data = pd.DataFrame()

# Loop through each row in the rows list
for row in rows:
    # Sort the bounding boxes of the cells in the row from left to right
    sorted_row = sorted(row, key=lambda x: x[0])

    # Extract the cell data from the sorted bounding boxes
    cell_data = []
    for bb in sorted_row:
        xmin, ymin, w, h = bb
        cell_image = tbl_image[ymin:ymin+h, xmin:xmin+w]
        cell_text = "Extracted cell data" # Replace this with your code to extract the text from the cell image
        cell_data.append(cell_text)
    
    # Add the extracted cell data as a row to the table_data dataframe
    table_data = table_data.append(pd.Series(cell_data), ignore_index=True)

# Print the final dataframe
print(table_data)



In [ ]:
boundingBoxes

In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# demo document
dem = dict(
    img_path='image1.png',
    gt_boxes=np.array([[451,  67, 749, 749]]),
    in_boxes=np.array([[455,  84, 760, 785]]))

dem_wdth = dem['in_boxes'][0][3] - dem['in_boxes'][0][1]
dem_hght = dem['in_boxes'][0][2] - dem['in_boxes'][0][0]
dem_xmin = dem['in_boxes'][0][1]
dem_ymin = dem['in_boxes'][0][0]

dem_image = cv2.imread(dem['img_path'])

# detected table from document
tbl_image = dem_image[dem_ymin: dem_ymin + dem_hght,
                      dem_xmin: dem_xmin + dem_wdth]

# threshold and resize table image
tbl_gray = cv2.cvtColor(tbl_image, cv2.COLOR_BGR2GRAY)
tbl_thresh_bin = cv2.threshold(tbl_gray, 127, 255, cv2.THRESH_BINARY)[1]

R = 2.5
tbl_resized = resize(tbl_thresh_bin, width=int(tbl_image.shape[1] // R))


def get_dividers(img, axis):
    """Return array indicies of white horizontal or vertical lines."""
    blank_lines = np.where(np.all(img == 255, axis=axis))[0]
    filtered_idx = np.where(np.diff(blank_lines) != 1)[0]
    return blank_lines[filtered_idx]


dims = tbl_image.shape[0], tbl_image.shape[1]

# table mask to search for gridlines
tbl_str = np.zeros(dims, np.uint8)
tbl_str = cv2.rectangle(tbl_str, (0, 0), (dims[1] - 1, dims[0] - 1), 255, 1)

for a in [0, 1]:
    dividers = get_dividers(tbl_resized, a)
    start_point = [0, 0]
    end_point = [dims[1], dims[1]]
    for i in dividers:
        i *= R
        start_point[a] = int(i)
        end_point[a] = int(i)
        cv2.line(tbl_str,
                 tuple(start_point),
                 tuple(end_point),
                 255,
                 1)


contours, hierarchy = cv2.findContours(tbl_str,
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)


def sort_contours(cnts, method="left-to-right"):
    """Return sorted countours."""
    reverse = False
    k = 0
    if method in ['right-to-left', 'bottom-to-top']:
        reverse = True
    if method in ['top-to-bottom', 'bottom-to-top']:
        k = 1
    b_boxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, b_boxes) = zip(*sorted(zip(cnts, b_boxes),
                                  key=lambda b: b[1][k],
                                  reverse=reverse))
    return (cnts, b_boxes)


contours, boundingBoxes = sort_contours(contours, method='top-to-bottom')

# remove countours of the whole table
bb_filtered = [list(t) for t in boundingBoxes
               if t[2] < dims[1] and t[3] < dims[0]]

# allocate countours in table-like structure
rows = []
columns = []

for i, bb in enumerate(bb_filtered):
    if i == 0:
        columns.append(bb)
        previous = bb
    else:
        if bb[1] < previous[1] + previous[3]/2:
            columns.append(bb)
            previous = bb
            if i == len(bb_filtered) - 1:
                rows.append(columns)
        else:
            rows.append(columns)
            columns = []
            previous = bb
            columns.append(bb)

import pandas as pd

# Create an empty dataframe to store the final table data
table_data = pd.DataFrame()

# Loop through each row in the rows list
for row in rows:
    # Sort the bounding boxes of the cells in the row from left to right
    sorted_row = sorted(row, key=lambda x: x[0])

    # Extract the cell data from the sorted bounding boxes
    cell_data = []
    for bb in sorted_row:
        xmin, ymin, w, h = bb
        cell_image = tbl_image[ymin:ymin+h, xmin:xmin+w]
        cell_text = pytesseract.image_to_string(cell_image)
        cell_data.append(cell_text)
    
    # Add the extracted cell data as a row to the table_data dataframe
    table_data = table_data.append({'Column1': cell_data[0], 'Column2': cell_data[1], 'Column3': cell_data[2], 'Column4': cell_data[3]}, ignore_index=True)

# Print the table data
print(table_data)

In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd
import pytesseract

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# Function to read tabular data from an image and store it in Excel
def read_tabular_data_from_image(image_path):

    # Read the image
    try:
        image = cv2.imread(image_path)
    except FileNotFoundError:
        print('The image file does not exist.')
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding to binarize the image
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Dilate the binary image to fill in small holes
    dilated = cv2.dilate(thresh, np.ones((3, 3), np.uint8))

    # Find contours in the dilated image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by their area, in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Extract the largest contour, which is the table
    table_contour = contours[0]

    # Get the bounding box of the table
    (x, y, w, h) = cv2.boundingRect(table_contour)

    # Extract the table from the image
    table = image[y:y + h, x:x + w]

    # Resize the table to fit on a single page
    table = resize(table, width=1000)

    # Extract the text from each cell in the table
    cell_text = []
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            cell_image = table[i:i + 1, j:j + 1]
            cell_text.append(pytesseract.image_to_string(cell_image))

    # Create a Pandas DataFrame with the extracted text
    table_data = pd.DataFrame(cell_text)

    # Write the DataFrame to Excel
    table_data.to_excel('table_data.xlsx', index=False)

    return table_data


# If this code is being run as the main script, read the image from the command line and store the tabular data in Excel
if __name__ == '__main__':

    # Get the image path from the command line
    image_path = input('Enter the path to the image: ')

    # Read the tabular data from the image and store it in Excel
    table_data = read_tabular_data_from_image(image_path)

    # Print the table data
    print(table_data)


In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd
import pytesseract

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# Function to read tabular data from an image and store it in Excel
def read_tabular_data_from_image(image_path):

    # Read the image
    try:
        image = cv2.imread(image_path)
    except FileNotFoundError:
        print('The image file does not exist.')
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding to binarize the image
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Dilate the binary image to fill in small holes
    dilated = cv2.dilate(thresh, np.ones((3, 3), np.uint8))

    # Find contours in the dilated image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by their area, in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Extract the largest contour, which is the table
    table_contour = contours[0]

    # Get the bounding box of the table
    (x, y, w, h) = cv2.boundingRect(table_contour)

    # Extract the table from the image
    table = image[y:y + h, x:x + w]

    # Resize the table to fit on a single page
    table = resize(table, width=1000)

    # Extract the text from each cell in the table
    cell_text = []
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            cell_image = table[i:i + 1, j:j + 1]
            cell_text.append(pytesseract.image_to_string(cell_image))

    # Create a Pandas DataFrame with the extracted text
    table_data = pd.DataFrame(cell_text)

    # Clean the DataFrame by removing empty cells and cells with only whitespace
    table_data = table_data.dropna(how='all')
    table_data = table_data[table_data.str.len() > 0]

    # Rename the columns of the DataFrame
    table_data.columns = ['Column' + str(i) for i in range(table_data.shape[1])]

    # Return the DataFrame
    return table_data


# If this code is being run as the main script, read the image from the command line and store the tabular data in Excel
if __name__ == '__main__':

    # Get the image path from the command line
    image_path = input('Enter the path to the image: ')

    # Read the tabular data from the image and store it in Excel
    table_data = read_tabular_data_from_image(image_path)

    # Print the table data
    print(table_data)

In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd
import pytesseract

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# Function to read tabular data from an image and store it in Excel
def read_tabular_data_from_image(image_path):

    # Read the image
    try:
        image = cv2.imread(image_path)
    except FileNotFoundError:
        print('The image file does not exist.')
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding to binarize the image
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Dilate the binary image to fill in small holes
    dilated = cv2.dilate(thresh, np.ones((3, 3), np.uint8))

    # Find contours in the dilated image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by their area, in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Extract the largest contour, which is the table
    table_contour = contours[0]

    # Get the bounding box of the table
    (x, y, w, h) = cv2.boundingRect(table_contour)

    # Extract the table from the image
    table = image[y:y + h, x:x + w]

    # Resize the table to fit on a single page
    table = resize(table, width=1000)

    # Extract the text from each cell in the table
    cell_text = []
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            cell_image = table[i:i + 1, j:j + 1]
            cell_text.append(pytesseract.image_to_string(cell_image))

    # Create a Pandas DataFrame with the extracted text
    table_data = pd.DataFrame(cell_text)

    # Clean the DataFrame by removing empty cells and cells with only whitespace
    table_data = table_data.dropna(how='all')
    table_data = table_data[table_data.str.len() > 0]

    # Rename the columns of the DataFrame
    table_data.columns = ['Column' + str(i) for i in range(table_data.shape[1])]

    # Return the DataFrame
    return table_data


# If this code is being run as the main script, read the image from the command line and store the tabular data in Excel
if __name__ == '__main__':

    # Get the image path from the command line
    image_path = input('Enter the path to the image: ')

    # Read the tabular data from the image and store it in Excel
    table_data = read_tabular_data_from_image(image_path)

    # Print the table data
    print(table_data)

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import cv2
import numpy as np
import pandas as pd

# Global variables to store selected areas
column_one_area = None
column_two_area = None
heading_area = None

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image, column_one_area, column_two_area, heading_area

    # Open a file dialog to select an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    image_pil = Image.open(image_path)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

    # Reset the selected areas
    column_one_area = None
    column_two_area = None
    heading_area = None

# Function to reset the selected areas
def reset_selections():
    global column_one_area, column_two_area, heading_area

    # Set all areas to None
    column_one_area = None
    column_two_area = None
    heading_area = None

    # Update display
    update_display()

# Function to select column one area
def select_column_one_area():
    global column_one_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_one_area_click)

def on_column_one_area_click(event):
    global column_one_area

    # Save the starting point of the selection
    column_one_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_one_area_release)

def on_column_one_area_release(event):
    global column_one_area

    # Save the ending point of the selection
    column_one_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select column two area
def select_column_two_area():
    global column_two_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_two_area_click)

def on_column_two_area_click(event):
    global column_two_area

    # Save the starting point of the selection
    column_two_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_two_area_release)

def on_column_two_area_release(event):
    global column_two_area

    # Save the ending point of the selection
    column_two_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select heading area
def select_heading_area():
    global heading_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_heading_area_click)

def on_heading_area_click(event):
    global heading_area

    # Save the starting point of the selection
    heading_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_heading_area_release)

def on_heading_area_release(event):
    global heading_area

    # Save the ending point of the selection
    heading_area += (event.x, event.y)

    # Update display
    update_display()

# Function to update the display with selected areas
def update_display():
    global image_pil, column_one_area, column_two_area, heading_area

    # Create a copy of the original PIL image
    image_display = image_pil.copy()

    # Draw rectangles for selected areas on the image
    if column_one_area is not None:
        draw_rectangle(image_display, column_one_area)
    if column_two_area is not None:
        draw_rectangle(image_display, column_two_area)
    if heading_area is not None:
        draw_rectangle(image_display, heading_area)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_display)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

# Function to draw a rectangle on an image
def draw_rectangle(image, area):
    # Get the width and height of the rectangle
    width = area[2] - area[0]
    height = area[3] - area[1]

    # Create a rectangle object
    rect = ImageDraw.Draw(image)

    # Draw the rectangle on the image
    rect.rectangle((area[0], area[1], area[2], area[3]))

# Main function
def main():
    # Open an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    image_pil = Image.open(image_path)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Create a label for displaying the image
    label_image = tk.Label(root, image=image_tk)
    label_image.pack()

    # Create buttons
    button_column_one = tk.Button(root, text="Column One")
    button_column_one.pack()
    button_column_two = tk.Button(root, text="Column Two")
    button_column_two.pack()
    button_heading = tk.Button(root, text="Heading")
    button_heading.pack()

    # Bind the mouse button click events to the appropriate functions
    label_image.bind("<Button 1>", on_column_one_area_click)
    label_image.bind("<Button 1>", on_column_two_area_click)
    label_image.bind("<Button 1>", on_heading_area_click)

    # Start the main loop
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import cv2
import numpy as np
import pandas as pd

# Global variables to store selected areas
column_one_area = None
column_two_area = None
heading_area = None

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image, column_one_area, column_two_area, heading_area

    # Open a file dialog to select an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    with open(image_path, "rb") as f:
        image_pil = Image.open(f)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

    # Reset the selected areas
    column_one_area = None
    column_two_area = None
    heading_area = None

# Function to reset the selected areas
def reset_selections():
    global column_one_area, column_two_area, heading_area

    # Set all areas to None
    column_one_area = None
    column_two_area = None
    heading_area = None

    # Update display
    update_display()

# Function to select column one area
def select_column_one_area():
    global column_one_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_one_area_click)

def on_column_one_area_click(event):
    global column_one_area

    # Save the starting point of the selection
    column_one_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_one_area_release)

def on_column_one_area_release(event):
    global column_one_area

    # Save the ending point of the selection
    column_one_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select column two area
def select_column_two_area():
    global column_two_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_two_area_click)

def on_column_two_area_click(event):
    global column_two_area

    # Save the starting point of the selection
    column_two_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_two_area_release)

def on_column_two_area_release(event):
    global column_two_area

    # Save the ending point of the selection
    column_two_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select heading area
def select_heading_area():
    global heading_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_heading_area_click)

def on_heading_area_click(event):
    global heading_area

    # Save the starting point of the selection
    heading_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_heading_area_release)

def on_heading_area_release(event):
    global heading_area

    # Save the ending point of the selection
    heading_area += (event.x, event.y)

    # Update display
    update_display()

# Function to update the display with selected areas
def update_display():
    global image_pil, column_one_area, column_two_area, heading_area

    # Create a copy of the original PIL image
    image_display = image_pil.copy()

    # Draw rectangles for selected areas on the image
    if column_one_area is not None:
        draw_rectangle(image_display, column_one_area)
    if column_two_area is not None:
        draw_rectangle(image_display, column_two_area)
    if heading_area is not None:
        draw_rectangle(image_display, heading_area)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_display)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

# Function to draw a rectangle on an image
def draw_rectangle(image, area):
    # Get the width and height of the rectangle
    width = area[2] - area[0]
    height = area[3] - area[1]

    # Create a rectangle object
    rect = ImageDraw.Draw(image)

    # Draw the rectangle on the image
    rect.rectangle((area[0], area[1], area[2], area[3]))

# Main function
def main():
    # Open an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    with open(image_path, "rb") as f:
        image_pil = Image.open(f)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Create a label for displaying the image
    label_image = tk.Label(root, image=image_tk)
    label_image.pack()

    # Create buttons
    button_column_one = tk.Button(root, text="Column One")
    button_column_one.pack()
    button_column_two = tk.Button(root, text="Column Two")
    button_column_two.pack()
    button_heading = tk.Button(root, text="Heading")
    button_heading.pack()

    # Bind the mouse button click events to the appropriate functions
    label_image.bind("<Button 1>", on_column_one_area_click)
    label_image.bind("<Button 1>", on_column_two_area_click)
    label_image.bind("<Button 1>", on_heading_area_click)

    # Start the main loop
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
import pandas as pd
from pytesseract import pytesseract

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image

    # Open a file dialog to select an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    with open(image_path, "rb") as f:
        image_pil = Image.open(f)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

# Function to perform opencv and ocr on the tabular data
def process_image():
    global image_pil

    # Convert the PIL image to a NumPy array
    image_np = np.array(image_pil)

    # Apply opencv to the image
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Apply ocr to the image
    text = pytesseract.image_to_string(thresh)

    # Create a DataFrame from the text
    df = pd.DataFrame(text.split('\n'), columns=['Column 1', 'Column 2'])

    # Generate an excel file
    df.to_excel('output.xlsx')

# Create a button to open an image file
button_open_image = tk.Button(root, text="Open Image")
button_open_image.pack()

# Bind the button click event to the open_image function
button_open_image.config(command=open_image)

# Create a button to process the image
button_process_image = tk.Button(root, text="Process Image")
button_process_image.pack()

# Bind the button click event to the process_image function
button_process_image.config(command=process_image)

# Start the main loop
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
import pandas as pd
from pytesseract import pytesseract

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image

    # Open a file dialog to select an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    with open(image_path, "rb") as f:
        image_pil = Image.open(f)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

# Function to perform opencv and ocr on the tabular data
def process_image():
    global image_pil

    # Convert the PIL image to a NumPy array
    image_np = np.array(image_pil)

    # Apply opencv to the image
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Apply ocr to the image
    text = pytesseract.image_to_string(thresh)

    # Create a DataFrame from the text
    df = pd.DataFrame(text.split('\n'), columns=['Column 1', 'Column 2'])

    # Generate an excel file
    df.to_excel('output.xlsx')

# Create a button to open an image file
button_open_image = tk.Button(root, text="Open Image")
button_open_image.pack()

# Bind the button click event to the open_image function
button_open_image.config(command=open_image)

# Create a button to process the image
button_process_image = tk.Button(root, text="Process Image")
button_process_image.pack()

# Bind the button click event to the process_image function
button_process_image.config(command=process_image)

# Start the main loop
root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
import pandas as pd
from pytesseract import pytesseract

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image

    # Open a file dialog to select an image file
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    with open(image_path, "rb") as f:
        image_pil = Image.open(f)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

# Function to perform opencv and ocr on the tabular data
def process_image():
    global image_pil

    # Convert the PIL image to a NumPy array
    image_np = np.array(image_pil)

    # Apply opencv to the image
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Apply ocr to the image
    text = pytesseract.image_to_string(thresh)

    # Create a DataFrame from the text
    df = pd.DataFrame(text.split('\n'), columns=['Column 1', 'Column 2'])

    # Generate an excel file
    df.to_excel('output.xlsx')

# Create a button to open an image file
button_open_image = tk.Button(root, text="Open Image")
button_open_image.pack()

# Bind the button click event to the open_image function
button_open_image.config(command=open_image)

# Create a button to process the image
button_process_image = tk.Button(root, text="Process Image")
button_process_image.pack()

# Bind the button click event to the process_image function
button_process_image.config(command=process_image)

# Start the main loop
root.mainloop()

In [ ]:
import streamlit as st
import cv2
import numpy as np
import pandas as pd
from pytesseract import pytesseract

# Set the title of the app
st.title("OCR Tool")

# Add a select box to choose the language
language = st.selectbox("Choose a language", ["English", "French", "Spanish", "German", "Italian"])

# Add a file upload button
image = st.file_uploader("Upload an image")

# If an image is uploaded
if image is not None:

    # Convert the image to a NumPy array
    image_np = np.array(image)

    # Apply opencv to the image
    gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Apply ocr to the image
    text = pytesseract.image_to_string(thresh, lang=language)

    # Display the text
    st.write(text)

In [ ]:
pip install streamlit


In [ ]:
import numpy as np
import cv2
from imutils import resize
import pandas as pd
import pytesseract

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'


# Function to read tabular data from an image and store it in Excel
def read_tabular_data_from_image(image_path):

    # Read the image
    try:
        image = cv2.imread(image_path)
    except FileNotFoundError:
        print('The image file does not exist.')
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding to binarize the image
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Dilate the binary image to fill in small holes
    dilated = cv2.dilate(thresh, np.ones((3, 3), np.uint8))

    # Find contours in the dilated image
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by their area, in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Extract the largest contour, which is the table
    table_contour = contours[0]

    # Get the bounding box of the table
    (x, y, w, h) = cv2.boundingRect(table_contour)

    # Extract the table from the image
    table = image[y:y + h, x:x + w]

    # Resize the table to fit on a single page
    table = resize(table, width=1000)

    # Extract the text from each cell in the table
    cell_text = []
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            cell_image = table[i:i + 1, j:j + 1]
            cell_text.append(pytesseract.image_to_string(cell_image))

    # Create a Pandas DataFrame with the extracted text
    table_data = pd.DataFrame(cell_text)

    # Clean the DataFrame by removing empty cells and cells with only whitespace
    table_data = table_data.dropna(how='all')
    table_data = table_data[table_data[0].str.strip().astype(bool)]

    # Rename the columns of the DataFrame
    table_data.columns = ['Column' + str(i) for i in range(table_data.shape[1])]

    # Return the DataFrame
    return table_data


# If this code is being run as the main script, read the image path from the command line and store the tabular data in Excel
if __name__ == '__main__':

    # Get the image path from the command line
    image_path = input('Enter the path to the image: ')

    # Read the tabular data from the image and store it in Excel
    table_data = read_tabular_data_from_image(image_path)

    # Check if table data is not None
    if table_data is not None:
        # Store the tabular data in an Excel file
        table_data.to_excel('table_data.xlsx', index=False)
        print('Tabular data saved to table_data.xlsx')
    else:
        print('Failed to extract tabular data from the image.')


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import cv2
import numpy as np
import pandas as pd

# Global variables to store selected areas
column_one_area = None
column_two_area = None

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()

# Function to open an image file
def open_image():
    global image_tk, image_pil, label_image, column_one_area, column_two_area

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    image_pil = Image.open(file_path)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

    # Reset the selected areas
    column_one_area = None
    column_two_area = None

# Function to reset the selected areas
def reset_selections():
    global column_one_area, column_two_area

    # Set all areas to None
    column_one_area = None
    column_two_area = None

    # Update display
    update_display()

# Function to select column one area
def select_column_one_area():
    global column_one_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_one_area_click)

def on_column_one_area_click(event):
    global column_one_area

    # Save the starting point of the selection
    column_one_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_one_area_release)

def on_column_one_area_release(event):
    global column_one_area

    # Save the ending point of the selection
    column_one_area += (event.x, event.y)

    # Update display
    update_display()

# Function to select column two area
def select_column_two_area():
    global column_two_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_two_area_click)

def on_column_two_area_click(event):
    global column_two_area

    # Save the starting point of the selection
    column_two_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_two_area_release)

def on_column_two_area_release(event):
    global column_two_area

    # Save the ending point of the selection
    column_two_area += (event.x, event.y)

    # Update display
    update_display()

# Function to update the display with selected areas
def update_display():
    global image_pil, column_one_area, column_two_area

    # Create a copy of the original PIL image
    image_display = image_pil.copy()

    # Draw rectangles for selected areas on the image
    if column_one_area is not None:
        draw_rectangle(image_display, column_one_area)
    if column_two_area is not None:
        draw_rectangle(image_display, column_two_area)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_display)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)
    label_image.image = image_tk  # Keep a reference to avoid garbage collection

# Function to draw a rectangle on an image
def draw_rectangle(image, coords):
    draw = ImageDraw.Draw(image)
    draw.rectangle(coords, outline='red', width=2)

# Function to perform OCR on selected areas
def perform_ocr():
    global image_pil, column_one_area, column_two_area

    # Check if both areas are selected
    if column_one_area is None or column_two_area is None:
        messagebox.showerror("Error", "Please select both areas before performing OCR.")
        return

    # Crop selected areas from the original image
    column_one_image = image_pil.crop(column_one_area)
    column_two_image = image_pil.crop(column_two_area)

    # Perform OCR on column one image
    column_one_text = pytesseract.image_to_string(column_one_image)

    # Perform OCR on column two image
    column_two_text = pytesseract.image_to_string(column_two_image)

    # Display the extracted text in a message box
    result = f"Column One Text:\n{column_one_text}\n\nColumn Two Text:\n{column_two_text}"
    messagebox.showinfo("OCR Result", result)

# Create buttons for various actions
open_button = tk.Button(root, text="Open Image", command=open_image)
open_button.pack()

reset_button = tk.Button(root, text="Reset Selections", command=reset_selections)
reset_button.pack()

select_column_one_button = tk.Button(root, text="Select Column One Area", command=select_column_one_area)
select_column_one_button.pack()

select_column_two_button = tk.Button(root, text="Select Column Two Area", command=select_column_two_area)
select_column_two_button.pack()

perform_ocr_button = tk.Button(root, text="Perform OCR", command=perform_ocr)
perform_ocr_button.pack()

# Run the tkinter event loop
root.mainloop()


In [ ]:
import cv2
import numpy as np
from pytesseract import pytesseract

# Load the image
image = cv2.imread("image.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the image
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

# Apply OCR to the image
text = pytesseract.image_to_string(thresh)

# Display the text
print(text)


In [ ]:
import cv2
import numpy as np
from pytesseract import pytesseract

# Load the image
image = cv2.imread("image.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the image
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

# Apply OCR to the image
text = pytesseract.image_to_string(thresh, config='--oem 3 --psm 7')

# Split the text into lines
lines = text.split('\n')

# Create a dataframe
df = pd.DataFrame(lines, columns=['Row 1', 'Row 2'])

# Display the dataframe
print(df)

In [ ]:
import cv2
import numpy as np
from pytesseract import pytesseract
from PIL import Image

# Load the image
image = Image.open("image.png")

# Convert the image to RGB format
image = image.convert("RGB")

# Save the image in JPG format
image.save("image.jpg")

# Load the image
image = cv2.imread("image.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the image
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

# Apply OCR to the image
text = pytesseract.image_to_string(thresh, config='--oem 3 --psm 7')

# Split the text into lines
lines = text.split('\n')

# Create a dataframe
df = pd.DataFrame(lines, columns=['Row 1', 'Row 2'])

# Display the dataframe
print(df)

In [ ]:
import cv2
import numpy as np
from pytesseract import pytesseract
from PIL import Image

# Load the image
image = Image.open("image.png")

# Convert the image to RGB format
image = image.convert("RGB")

# Save the image in JPG format
image.save("image.jpg")

# Load the image
image = cv2.imread("image.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the image
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

# Apply OCR to the image
text = pytesseract.image_to_string(thresh, config='--oem 3 --psm 7')

# Split the text into lines
lines = text.split('\n')

# Create a dataframe
df = pd.DataFrame(lines, columns=['Row 1', 'Row 2'])

# Display the dataframe
print(df)

In [10]:
import cv2
import numpy as np
import pytesseract
import pandas as pd

# Set the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'

# Load the image
image = Image.open("image.png")

# Convert the image to a NumPy array
image = np.array(image)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to the image
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

# Apply OCR to the image
text = pytesseract.image_to_string(thresh, config='--oem 3 --psm 7')

# Split the text into lines
lines = text.split('\n')

# Create a dataframe
df = pd.DataFrame()

# Iterate over the lines
for line in lines:

    # Split the line into cells
    cells = line.split('|')

    # Create a row for each cell
    for cell in cells:
        df = df.append({
            "Cell": cell
        }, ignore_index=True)

# Display the dataframe
print(df)

  Cell
0     
